In [ ]:
import numpy as np
import os
import sys
import string
import pandas as pd
import re

# Add these to the string punctuation, since they show up in strings here
string.punctuation+="–☐■"


In [ ]:
Base="NF"
Bases=os.listdir("txt/")

# The shortest plausible title
lencut=10

# These words indicate that the sentence is not yet over
HangingWords=["of","for","at","the","with","from", " by", "using","and","neutrino","other","new","beyond","after","above","without"," a"," in","-",":","–","next","high"]

# These are words that indicate this may not be the title
RemoveWords=["snowmass","2021","letter","of","interest","submission","submitted","to","august","2020","topical","group","working","intent","process","contribution","21"]

# And these will be removed from whatever we ultimately extract as the title
SkipPatterns = [re.compile("Snowmass",re.IGNORECASE),
            re.compile("2021",re.IGNORECASE),
            re.compile("Snowmass-2021",re.IGNORECASE),
            re.compile("Snowmass2021",re.IGNORECASE),
            re.compile("Letter Of Interest", re.IGNORECASE),
            re.compile("Letter Of Intent", re.IGNORECASE),
            re.compile("LOI", re.IGNORECASE),
            re.compile(":", re.IGNORECASE)]

SkipWords=RemoveWords+HangingWords
# How many lines of parsed pdf text will we realistically mine before finding the title
maxline=40 


In [ ]:
Titles=[]
FileNames=[]
Frontiers=[]



#loop over frontiers
for Base in Bases:
    Files=os.listdir("txt/"+Base)

    length=len(Files)
    
    # loop over files
    for n in range(0,length):
        Title=""

        #Open that text file
        f=open("txt/"+Base+"/"+Files[n])
        FullText= f.readlines()
        
        #Crawl lines of text
        line=0
        while(line<maxline):
            try:
                rawstring=FullText[line]
                
                # This line might be the title if it contains some non-generic words.
                #  Lets remove the generic ones, whitespace and punctuation and see whats left.
                workstring=rawstring.lower()
                workstring=' '.join(workstring.split())
                for r in SkipWords:
                    workstring=workstring.replace(r,"")
                workstring=workstring.translate(str.maketrans('', '', string.punctuation)).strip()
                if(len(workstring)>lencut):
                    Title=rawstring.strip()
                    
                    # if title ends on a hanging word, go get the next line and add it on.
                    #  This is easy-ish. General detection of multi-line titles is not.
                    HangingTitle=True
                    while(HangingTitle==True):
                        HangingTitle=False
                        for i in HangingWords:
                            if(Title[-len(i):].lower()==i):            
                                HangingTitle=True
                        if(HangingTitle==True):
                            line=line+1
                            Title+=" " + FullText[line]
                            Title=Title.strip()
                            if(line>maxline):
                                break
                    break
            except:
                break  # If I fails I bails.
            else:
                line=line+1
    
        #Remove Snowmass and LOI patterns
        Title=' '.join(Title.split())
        for p in patterns:
            Title=p.sub("", Title).strip()
        
        # Remove leading and trailing punctuation and whitespace
        Title=Title.strip(string.punctuation).strip()
        
        # Add to data table
        Titles.append(Title)
        FileNames.append(Files[n][0:-4])
        Frontiers.append(Base)

        print(Title)


In [ ]:
#Save it
dat=pd.DataFrame({"Frontier":Frontiers,"FileName":FileNames,"Title":Titles})
dat.to_excel("Titles.xlsx")
print(len(Titles))

In [ ]:
# Merge with the original master sheet
datmaster=pd.read_excel("LOIMaster.xlsx")
datmaster=datmaster.set_index("filename")
dat2=dat.set_index("FileName").drop("Frontier",axis=1)
datjoined=dat2.join(datmaster)
datjoined.insert(1,"FileName",datjoined.index)
datjoined=datjoined.set_index("number")
datjoined=datjoined[datjoined.frontier.notna()]

In [ ]:
# Save it
datjoined.to_excel("LOIMasterWithTitles.xlsx")